In [32]:
import requests
import pandas as pd
import pyarrow as pa
import boto3
from datetime import datetime
import os

In [2]:
link2 = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [3]:
link2

'https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad'

In [4]:
requisicao = requests.get(link2)

In [5]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03n'}],
 'base': 'stations',
 'main': {'temp': 289.14,
  'feels_like': 288.94,
  'temp_min': 288.7,
  'temp_max': 290.23,
  'pressure': 1020,
  'humidity': 82,
  'sea_level': 1020,
  'grnd_level': 1016},
 'visibility': 10000,
 'wind': {'speed': 1.35, 'deg': 66, 'gust': 2.19},
 'clouds': {'all': 29},
 'dt': 1697344512,
 'sys': {'type': 2,
  'id': 2020383,
  'country': 'BR',
  'sunrise': 1697359613,
  'sunset': 1697405610},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [6]:
df = requisicao.json()

In [7]:
df1 = pd.json_normalize(df)

In [8]:
df_lon_lat = pd.DataFrame(df['coord'], index=[0])

In [9]:
df_lon_lat

,lon,lat
0,-51.1306,-29.6783


In [10]:
df_weather = pd.DataFrame(df['weather'], index=[0])

In [11]:
df_weather

,id,main,description,icon
0,802,Clouds,scattered clouds,03n


In [12]:
df_main = pd.DataFrame(df['main'], index=[0])

In [13]:
df_main

,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level
0,289.14,288.94,288.7,290.23,1020,82,1020,1016


In [14]:
df1 = pd.DataFrame(df1)

In [15]:
df1

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 802, 'main': 'Clouds', 'description': ...",stations,10000,1697344512,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1016,1.35,66,2.19,29,2,2020383,BR,1697359613,1697405610


In [16]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [17]:
df_clumns_selected = df1[selected_columns]

In [18]:
df_clumns_selected

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1697344512,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,289.14,...,1016,1.35,66,2.19,29,2,2020383,BR,1697359613,1697405610


In [19]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [20]:
super_df.to_parquet("C:/Users/crist/Desktop/s3_testes/super_df.parquet")

In [21]:
super_df

,id,main,description,icon,lon,lat,temp,feels_like,temp_min,temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,802.0,Clouds,scattered clouds,03n,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-51.1306,-29.6783,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,289.14,288.94,288.7,290.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3456068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1016.0,1.35,66.0,2.19,29.0,2.0,2020383.0,BR,1.697360e+09,1.697406e+09


putaria com o boto3

In [22]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'

s3 = boto3.session.Session(profile_name=profile_name).client('s3')

In [23]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [24]:
local_file_path = "C:/Users/crist/Desktop/s3_testes/super_df.parquet"

In [63]:
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

In [64]:
dia_da_exucucao

'2023-10-15_02-03-06'

In [55]:
s3_file_name = f'df_teste_{dia_da_exucucao}.parquet'

In [56]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)

---------------------------------

In [57]:
data_e_hora_formatadas = dia_da_exucucao.strftime('%Y-%m-%d_%H-%M-%S')

AttributeError: 'str' object has no attribute 'strftime'

In [58]:

data_e_hora_atual = datetime.now()

# Formate a data e hora em um formato desejado (por exemplo, 'YYYY-MM-DD_HH-MM-SS')
data_e_hora_formatadas = data_e_hora_atual.strftime('%Y-%m-%d_%H-%M-%S')

# Crie o novo nome com a data e hora atual
novo_nome_do_dataframe = f'super_df_{data_e_hora_formatadas}'


In [59]:
novo_nome_do_dataframe

'super_df_2023-10-15_02-01-48'